#### BUILD A BASIC CHATBOT LANGGRAPH(GRAPH API)

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages:Annotated[list,add_messages]
    
    
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

llm=ChatGroq(model="llama3-8b-8192")

llm

llm=init_chat_model("groq:llama3-8b-8192")
llm

## Node Functionality
def chatbot(state:State):
    return {"messages":[llm.invoke(state["messages"])]}

graph_builder=StateGraph(State)

## Adding node
graph_builder.add_node("llmchatbot",chatbot)
## Adding Edges
graph_builder.add_edge(START,"llmchatbot")
graph_builder.add_edge("llmchatbot",END)

## compile the graph
graph=graph_builder.compile()


## Visualize the graph
from IPython.display import Image,display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass


response=graph.invoke({"messages":"Hi"})

In [ ]:
response["messages"][-1].content

In [ ]:
for event in graph.stream({"messages" : "Hi, How are you?"}):
    for val in event.values():
        print(val["messages"][-1].content)

#### CHATBOT INCLUDING TOOL

In [ ]:
from langchain_tavily import TavilySearch
tool = TavilySearch(max_results=2)
tool.invoke("what is langgraph")

In [ ]:
# Creating Custom function tool to bind with the llm model.
def multiply(a:int,b:int)->int:
    """Multiply a and b
    Args:
    a (int): first int
    b (int): second int 
    
    Returns:
    int: output int
    """
    return a*b

In [ ]:
tools = [tool,multiply]

In [ ]:
llm_tools = llm.bind_tools(tools)

In [ ]:
llm_tools

In [ ]:
## Stategraph
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

## Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_tools.invoke(state["messages"])]}

## Grpah
builder=StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition
)
builder.add_edge("tools",END)

## compile the graph
graph=builder.compile()

from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response = graph.invoke({"messages" : "what is recent AI news"})

In [ ]:
response["messages"][-1].content

In [ ]:
for m in response["messages"]:
    m.pretty_print()

In [ ]:
response = graph.invoke({"messages" : "what is 5 multiplied by 6 and then multiply by 4"})
for m in response["messages"]:
    m.pretty_print()

#### ReACT AGENT ARCHITECTURE

In [ ]:
## Stategraph
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

## Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_tools.invoke(state["messages"])]}

## Grpah
builder=StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition
)
builder.add_edge("tools","tool_calling_llm")

## compile the graph
graph=builder.compile()

from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response=graph.invoke({"messages":"Give me the recent ai news and then multiply 5 by 10"})
for m in response['messages']:
    m.pretty_print()

#### ADDING MEMORY IN AGENTIC GRAPH

In [ ]:
## Stategraph
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import tools_condition

memory = MemorySaver()

## Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_tools.invoke(state["messages"])]}

## Grpah
builder=StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition
)
builder.add_edge("tools","tool_calling_llm")

## compile the graph
graph=builder.compile(checkpointer=memory)

from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable":{"thread_id":"1"}}
response = graph.invoke({"messages":"Hi, my name is rohit"},config=config)

In [ ]:
response = graph.invoke({"messages":"what is my name?"},config=config)
print(response["messages"][-1].content)

#### STREAMING

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [ ]:
def superbot(state:State):
    return{"messages":[llm.invoke(state['messages'])]}

In [ ]:
graph = StateGraph(State)
graph.add_node("SuperBot",superbot)
graph.add_edge(START,"SuperBot")
graph.add_edge("SuperBot",END)
graph_builder = graph.compile(checkpointer=memory)

display(Image(graph_builder.get_graph().draw_mermaid_png()))

In [ ]:
## Invocation

config = {"configurable": {"thread_id": "1"}}

graph_builder.invoke({'messages':"Hi,My name is Krish And I like cricket"},config)

Streaming
Methods: .stream() and astream()

These methods are sync and async methods for streaming back results.
Additional parameters in streaming modes for graph state

values : This streams the full state of the graph after each node is called.
updates : This streams updates to the state of the graph after each node is called.

In [ ]:
config = {"configurable":{"thread_id":"3"}}
for chunk in graph_builder.stream({'messages':"hi,my name is rohit and i like badminton"},config,stream_mode="values"):
    print(chunk)

In [ ]:
config = {"configurable": {"thread_id": "5"}}

async for event in graph_builder.astream_events({"messages":["Hi My name is Rohit and I like to play cricket"]},config,version="v2"):
    print(event)